In [1]:
import pandas as pd
from binance.client import Client
import asyncio
from binance import AsyncClient, BinanceSocketManager
import nest_asyncio
nest_asyncio.apply()
from sqlalchemy import create_engine
from keys import bkey, secret
import numpy as np 
import ta
import datetime as dt
from tqdm import tqdm

In [2]:
client = Client(bkey,secret,tld='us')
engine = create_engine('sqlite:///backtest_usd.db')

In [3]:
info = client.get_exchange_info()
symbols = [x['symbol'] for x in info['symbols']]
exclude = ['UP','DOWN','BEAR','BULL']
non_lev = [symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]
relevant = [symbol for symbol in non_lev if symbol.endswith('USD')]

In [4]:
def getminutedataWithManip(symbol,lookback):
    df= pd.DataFrame(client.get_historical_klines(symbol,'1m',lookback+ ' days ago UTC'))
    
    df= df.iloc[:,:5]
    df.columns = ['Time','Open','High','Low','Close']
    df[['Open','High','Low','Close']] = df[['Open','High','Low','Close']].astype(float)
    df.Time = pd.to_datetime(df.Time,unit='ms')
    df['EMA200'] = ta.trend.ema_indicator(df.Close,window=200)
    df['wf_Top_bool'] = np.where(
    df['High']==df['High'].rolling(9,center=True).max(),True,False)
    df['wf_top'] = np.where(
    df['High']==df['High'].rolling(9,center=True).max(),df['High'],None
    )
    df['wf_top'] = df['wf_top'].ffill()
    df.dropna(inplace=True)
    df['Buy'] = np.where((df.Close> df.wf_top) & (df.Close>df.EMA200),1,0)
    df['SL'] = np.where(df.Buy==1, df.Close-(df.Close-df.Low),0)
    df['TP'] = np.where(df.Buy==1, df.Close+(df.Close-df.Low)*1.5,0)
    return df


In [5]:
for coin in tqdm(relevant):
    try:
        getminutedataWithManip(coin,'30').to_sql(coin,engine,index=False)
    except:
        pass

100%|██████████████████████████████████████████| 82/82 [42:08<00:00, 30.83s/it]


In [6]:
def min_check(row):
    row['minute']=dt.datetime.fromtimestamp(dt.datetime.timestamp(row['Time'])).time().minute
    return  row
def returnCalc(profit,invest):
    for i in profit:
        change = 1+i
        fee = (invest*0.00075)
        invest = (change*invest)-fee
    return invest
def profits(df):
    buys, sells = [],[]
    for i in range(len(df)):
        if df.Buy.iloc[i]:
            buys.append(df.iloc[i].name)
            for j in range(len(df)-i):
                if df.TP.iloc[i] < df.Close.iloc[i+j] or \
                df.SL.iloc[i] > df.Close.iloc[i+j]:
                    sells.append(df.iloc[i+j].name)
                    break
    frame = pd.DataFrame([buys,sells]).T.dropna()
    frame.columns = ['Buys','Sells']
    actualtrades = frame[frame.Buys>frame.Sells.shift(1)]
    profits = (df.loc[actualtrades.Sells].Close.values - \
    df.loc[actualtrades.Buys].Close.values) / df.loc[actualtrades.Buys].Close.values
    return profits
def check_strat(stonks,interval=1,invest=1000):
    res = {'ticker':[],'final_profit':[],'base_pcnt_change':[],'gain':[]}
    for i in stonks:
        try:
            df = pd.read_sql(i,engine)
            df = df.apply(lambda row: min_check(row), axis=1)
            df = df[df['minute']%interval==0]
            res['ticker'].append(i)
            base = df.copy()
            profit = profits(df)
            rets = returnCalc(profit,invest)
            res['final_profit'].append(rets)
            res['gain'].append(((rets-invest)/invest))
            res['base_pcnt_change'].append(list((base.Close.pct_change()+1).cumprod())[-1])
        except:
            pass
    #print(res)
    df=pd.DataFrame(res)
    return df
#ints = [1,15,30,45,60]
ints = [1]
for i in tqdm(ints):
    final = check_strat(relevant,i,1000)
    final['base_pcnt_change'] = final['base_pcnt_change'] -1
    final['win_delta'] = final['gain'] - final['base_pcnt_change']
    final.set_index('ticker', inplace=True)
    final.sort_values('final_profit', ascending=False,inplace=True)
    final.to_sql(str(i)+': analysis',engine,index=True,if_exists='replace')

100%|██████████████████████████████████████████| 1/1 [21:25<00:00, 1285.24s/it]


In [135]:
res = {'ticker':[],'final_profit':[],'pcnt_change':[]}
df = pd.read_sql('ZENUSDT',engine)
df = df.apply(lambda row: min_check(row), axis=1)
df = df[df['minute']%interval==0]
res['ticker'].append('BTCUSDT')
base = df.copy()
profit = profits(df)
res['final_profit'].append(sum(profit))
#res['pcnt_change'].append(((base.Close.pct_change() +1).cumprod()[-1]))

AttributeError: 'DataFrame' object has no attribute 'appy'

In [130]:
profit
def returnCalc(profit,invest):
    for i in profit:
        change = 1+i
        fee = (invest*0.00075)
        invest = (change*invest)-fee
    return invest
returnCalc(profit,1000)

1501.931629367075

In [123]:
def min_check(row):
    row['minute']=dt.datetime.fromtimestamp(dt.datetime.timestamp(row['Time'])).time().minute
    return  row
df= df.apply(lambda row: min_check(row),axis=1)


In [133]:
df

,Time,Open,High,Low,Close,EMA200,wf_Top_bool,wf_top,Buy,SL,TP,minute
0,2021-11-30 09:06:00,116.859,117.060,116.859,117.060,114.195033,False,115.091,1,116.859,117.3615,6
1,2021-11-30 09:07:00,117.249,117.523,116.566,116.626,114.219222,True,117.523,0,0.000,0.0000,7
2,2021-11-30 09:08:00,116.570,116.599,116.112,116.112,114.238056,False,117.523,0,0.000,0.0000,8
3,2021-11-30 09:09:00,116.014,116.014,115.523,115.655,114.252155,False,117.523,0,0.000,0.0000,9
4,2021-11-30 09:10:00,115.805,115.805,115.094,115.094,114.260531,False,117.523,0,0.000,0.0000,10
...,...,...,...,...,...,...,...,...,...,...,...,...
42997,2021-12-30 05:43:00,61.072,61.072,61.072,61.072,60.946580,True,61.072,0,0.000,0.0000,43
42998,2021-12-30 05:44:00,61.072,61.072,61.072,61.072,60.947828,False,61.072,0,0.000,0.0000,44
42999,2021-12-30 05:45:00,61.072,61.072,61.072,61.072,60.949064,False,61.072,0,0.000,0.0000,45
43000,2021-12-30 05:46:00,61.072,61.072,61.072,61.072,60.950287,False,61.072,0,0.000,0.0000,46


In [140]:
final_copy = final.copy()
final_copy['base_pcnt_change'] = final_copy['base_pcnt_change'] -1
final_copy['win_delta'] = final_copy['gain'] - final_copy['base_pcnt_change']
final_copy.set_index('ticker', inplace=True)
final_copy.sort_values('final_profit', ascending=False,inplace=True)
final_copy[final_copy['gain']>0]


,final_profit,base_pcnt_change,gain,win_delta
ticker,,,,
BATUSDT,1536.578433,-0.236911,0.536578,0.773489
CRVUSDT,1308.943864,0.052552,0.308944,0.256392
YFIUSDT,1305.050467,0.003807,0.305050,0.301243
HNTUSDT,1264.873959,-0.062029,0.264874,0.326903
AAVEUSDT,1263.165491,-0.053400,0.263165,0.316566
UNIUSDT,1256.624618,-0.153424,0.256625,0.410048
QTUMUSDT,1237.244591,-0.447614,0.237245,0.684859
EGLDUSDT,1229.070721,-0.379231,0.229071,0.608301
OXTUSDT,1195.323697,-0.261112,0.195324,0.456436


In [152]:
final_copy= final_copy[final_copy['gain']>0]

(sum(final_copy.final_profit)-(len(final_copy)*1000))/(len(final_copy)*1000)*100

13.348015202636606

In [149]:
(len(final_copy)*1000)

43000

In [176]:
ints = [1,15,30,45,60]
df1,df15,df30,df45,df60= pd.read_sql(str(1)+': analysis',engine),\
pd.read_sql(str(15)+': analysis',engine),pd.read_sql(str(30)+': analysis',engine),\
pd.read_sql(str(45)+': analysis',engine),pd.read_sql(str(60)+': analysis',engine)
dfs = [df1,df15,df30,df45,df60]


    
    

df_final = reduce(lambda left,right: pd.merge(left,right,on='ticker',), dfs)

In [169]:
from functools import reduce

In [178]:
df_final.columns

Index(['ticker', 'final_profit_x', 'base_pcnt_change_x', 'gain_x',
       'win_delta_x', 'final_profit_y', 'base_pcnt_change_y', 'gain_y',
       'win_delta_y', 'final_profit_x', 'base_pcnt_change_x', 'gain_x',
       'win_delta_x', 'final_profit_y', 'base_pcnt_change_y', 'gain_y',
       'win_delta_y', 'final_profit', 'base_pcnt_change', 'gain', 'win_delta'],
      dtype='object')

In [183]:
res = pd.concat(dfs,keys=[1,15,30,45,60])

In [192]:
res.to_excel('res.xlsx')

In [13]:
final

,final_profit,base_pcnt_change,gain,win_delta
ticker,,,,
ICXUSD,1909.965740,-0.296629,0.909966,1.206595
ONEBUSD,1790.038637,-0.113886,0.790039,0.903925
SUSHIUSD,1777.488546,0.242496,0.777489,0.534993
CTSIUSD,1755.430599,-0.321529,0.755431,1.076960
REPUSD,1619.573298,-0.183009,0.619573,0.802583
HBARBUSD,1562.781549,-0.160942,0.562782,0.723724
REPBUSD,1514.698263,-0.185921,0.514698,0.700619
ZILUSD,1507.717381,-0.105023,0.507717,0.612740
EOSBUSD,1478.595997,-0.234587,0.478596,0.713183


In [12]:
pd.set_option('display.max_rows', 80)